In [7]:
### MLJar for Telecom Churn project

## Proprocesing steps

#Data Structures
import pandas as pd
import numpy as np

#Sklearn
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, precision_score, recall_score

#Load the data
df_train = pd.read_csv("/Users/I353375/Downloads/MLOps/airflow/Telecom_Churn/telecom_train.csv")
df_test = pd.read_csv("/Users/I353375/Downloads/MLOps/airflow/Telecom_Churn/telecom_test.csv")
sample = pd.read_csv("/Users/I353375/Downloads/MLOps/airflow/Telecom_Churn/telecom_sample.csv")
data_dict = pd.read_csv("/Users/I353375/Downloads/MLOps/airflow/Telecom_Churn/telecom_data_dictionary.csv")

#impute with zeroes on both df_train and df_test
nullvalues=df_train.isnull().mean()*100 
nullreplacezero_columns_train =  df_train.columns[df_train.columns.str.contains('_amt|count_|fb_user|_pck|total_|max_|_mou|_others')]
nullreplacezero_columns_test =  df_test.columns[df_test.columns.str.contains('_amt|count_|fb_user|_pck|total_|max_|_mou|_others')]

#After looking at the min values and other desrciptive stats about each of the columns 
# we can be certain about replacing NAs with zeroes in these columns
for col in nullreplacezero_columns_train:
    df_train[col] = df_train[col].replace(np.NaN,0.0)
    
for col in nullreplacezero_columns_test:
    df_test[col] = df_test[col].replace(np.NaN,0.0)

# getting rid of columns with just 1 unique value
single_val_cols_train = df_train.columns[df_train.nunique() == 1]
df_train.drop(single_val_cols_train,axis=1,inplace=True)

# getting rid of columns with just 1 unique value
single_val_cols_test = df_test.columns[df_test.nunique() == 1]
df_test.drop(single_val_cols_test,axis=1,inplace=True)

# Removing date columns because these wouldn't give any valuable insights about churn.
date_columns =  df_train.columns[df_train.columns.str.contains('date')]
df_train.drop(date_columns,axis=1,inplace=True)
df_test.drop(date_columns,axis=1,inplace=True)

# #Dropping the arpu columns because nullvalues are percentages are > 70%, and there is no correct way to impute these
# df_train.drop((nullvalues[nullvalues > 0].index).tolist(),axis=1, inplace= True)
# df_test.drop((nullvalues[nullvalues > 0].index).tolist(),axis=1, inplace= True)

# new column Total_Recharge_Amount_Data = av_rech_amt*total_rech_data for each month
df_train['total_rech_amt_data_6'] = df_train.av_rech_amt_data_6 * df_train.total_rech_data_6
df_train['total_rech_amt_data_7'] = df_train.av_rech_amt_data_7 * df_train.total_rech_data_7
df_train['total_rech_amt_data_8'] = df_train.av_rech_amt_data_8 * df_train.total_rech_data_8

df_test['total_rech_amt_data_6'] = df_test.av_rech_amt_data_6 * df_test.total_rech_data_6
df_test['total_rech_amt_data_7'] = df_test.av_rech_amt_data_7 * df_test.total_rech_data_7
df_test['total_rech_amt_data_8'] = df_test.av_rech_amt_data_8 * df_test.total_rech_data_8

df_train['total_avg_rech_amt_6_7'] = (df_train.total_rech_amt_6 + df_train.total_rech_amt_data_6 + df_train.total_rech_amt_7+ df_train.total_rech_amt_data_7)/2
df_test['total_avg_rech_amt_6_7'] = (df_test.total_rech_amt_6 + df_test.total_rech_amt_data_6 + df_test.total_rech_amt_7+ df_test.total_rech_amt_data_7)/2

## Some more columns to delete
list_total_ic_cols = df_train.columns[df_train.columns.str.contains('total_ic_mou|std_ic_mou|loc_ic_mou',regex=True)]
df_train.drop(list_total_ic_cols,axis=1,inplace=True)
list_total_ic_cols.tolist()

list_total_ic_cols = df_test.columns[df_test.columns.str.contains('total_ic_mou|std_ic_mou|loc_ic_mou',regex=True)]
df_test.drop(list_total_ic_cols,axis=1,inplace=True)
list_total_ic_cols.tolist()

list_total_og_cols = df_train.columns[df_train.columns.str.contains('total_og_mou|std_og_mou|loc_og_mou',regex=True)]
df_train.drop(list_total_og_cols,axis=1,inplace=True)
list_total_og_cols.tolist()

list_total_og_cols = df_test.columns[df_test.columns.str.contains('total_og_mou|std_og_mou|loc_og_mou',regex=True)]
df_test.drop(list_total_og_cols,axis=1,inplace=True)
list_total_og_cols.tolist()

##handeling outliers

def remove_outliers_iqr(df, column):
    # Calculate Q1 (25th percentile) and Q3 (99th percentile)
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.99)
    # Calculate IQR
    IQR = Q3 - Q1  
    # Calculate lower and upper bounds
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR   
    # Filter the DataFrame
    filtered_df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]   
    return filtered_df

df_train=remove_outliers_iqr(df_train,df_train.columns)

df_test=remove_outliers_iqr(df_test,df_test.columns)

## Apply the KNN Imputer to both train and test data to impute the Null values post removal of outliers 
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)  
imputer.fit(df_train)
X_imputed = imputer.transform(df_train)

df_train = pd.DataFrame(X_imputed, columns=df_train.columns)

imputer = KNNImputer(n_neighbors=5)  
imputer.fit(df_test)
X_imputed = imputer.transform(df_test)

df_test = pd.DataFrame(X_imputed, columns=df_test.columns)

## Filtering High-Value Customers.

high_value_filter = df_train.total_avg_rech_amt_6_7.quantile(0.7)
high_val_cust_df_train = df_train[df_train.total_avg_rech_amt_6_7 > high_value_filter]

high_value_filter = df_test.total_avg_rech_amt_6_7.quantile(0.7)
high_val_cust_df_test = df_test[df_test.total_avg_rech_amt_6_7 > high_value_filter]

df_train=high_val_cust_df_train.copy()

df_unseen_test=df_test.copy()

## MINMAX Scaling

from sklearn.preprocessing import MinMaxScaler
data2=df_train.drop(['id','churn_probability'],axis=1)
data3=df_test.drop('id',axis=1)
#initiate a object
scaler=MinMaxScaler()

## Create a list of Numerical vars
var1=data2.columns
var2=data3.columns
#Fit the method
df_train[var1]= scaler.fit_transform(df_train[var1])
df_test[var2]= scaler.transform(df_test[var2])

df_train['churn_probability']=df_train['churn_probability'].replace({'No churn': 0, 'Churn': 1})

df_train.id = df_train.id.astype(int)
df_train.churn_probability = df_train.churn_probability.astype(int)
df_test.id = df_test.id.astype(int)

df_unseen_test=df_test.copy()

## Train test Split

from sklearn.model_selection import train_test_split 
df_train,df_test = train_test_split(df_train, train_size=0.7,random_state=100)

y_train=df_train['churn_probability']
X_train=df_train.drop(['id','churn_probability'],axis=1)
 
y_test=df_test['churn_probability']
X_test=df_test.drop(['id','churn_probability'],axis=1)

## Lets use SMOTE to Remove the data imbalance
from imblearn.datasets import make_imbalance
from imblearn.under_sampling import NearMiss
from imblearn.pipeline import make_pipeline
from imblearn.metrics import classification_report_imbalanced

from imblearn.over_sampling import SMOTE
from collections import Counter


# oversampling the train dataset using SMOTE
smt = SMOTE()
X_train, y_train = smt.fit_resample(X_train, y_train)
X_test, y_test = smt.fit_resample(X_test, y_test)

# Save files with compression for faster I/O
X_train.to_csv('/Users/I353375/Downloads/MLOps/airflow/Telecom_Churn/X_train.csv', index=False)
X_test.to_csv('/Users/I353375/Downloads/MLOps/airflow/Telecom_Churn/X_test.csv', index=False)
y_train.to_csv('/Users/I353375/Downloads/MLOps/airflow/Telecom_Churn/y_train.csv', index=False)
y_test.to_csv('/Users/I353375/Downloads/MLOps/airflow/Telecom_Churn/y_test.csv', index=False)





/var/folders/7p/jmjy_99j6bx7_xbf0t58gz2m0000gn/T/ipykernel_32285/1432343492.py:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train['total_rech_amt_data_6'] = df_train.av_rech_amt_data_6 * df_train.total_rech_data_6
/var/folders/7p/jmjy_99j6bx7_xbf0t58gz2m0000gn/T/ipykernel_32285/1432343492.py:57: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train['total_rech_amt_data_7'] = df_train.av_rech_amt_data_7 * df_train.total_rech_data_7
/var/folders/7p/jmjy_99j6bx7_xbf0t58gz2m0000gn/T/ipykernel_32285/1432343492.py:58: Perform

In [9]:
### -- model bulding using ML Jar One time run to save the model in MlFlow

import os
import shutil
import mlflow
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from supervised.automl import AutoML
from mlflow.models.signature import infer_signature

# --- Step 1: MLflow Setup ---
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Telecom_Churn_Classification")
mlflow.autolog(disable=True)  # Manual logging

# --- Define paths ---
experiment_path = "/Users/I353375/Downloads/MLOps/mlflow_server/mljar_classification/experiments"
model_path = "/Users/I353375/Downloads/MLOps/mlflow_server/mljar_classification/models"
results_path = "/Users/I353375/Downloads/MLOps/mlflow_server/mljar_classification/automl_results"

os.makedirs(experiment_path, exist_ok=True)
os.makedirs(model_path, exist_ok=True)
os.makedirs(results_path, exist_ok=True)

# --- Step 2: Load Data ---
X_train = pd.read_csv('/Users/I353375/Downloads/MLOps/airflow/Telecom_Churn/X_train.csv')
X_test = pd.read_csv('/Users/I353375/Downloads/MLOps/airflow/Telecom_Churn/X_test.csv')
y_train = pd.read_csv('/Users/I353375/Downloads/MLOps/airflow/Telecom_Churn/y_train.csv')
y_test = pd.read_csv('/Users/I353375/Downloads/MLOps/airflow/Telecom_Churn/y_test.csv')

y_train.columns = ['churn_probability']
y_test.columns = ['churn_probability']

# --- Step 3: Train AutoML ---
automl = AutoML(
    mode="Compete",
    total_time_limit=600,
    eval_metric="logloss",  # 💡 Typical metric for classification
    explain_level=0,
    algorithms=["Xgboost", "LightGBM", "CatBoost", "Linear", "Decision Tree", "Extra Trees"],
    ml_task="binary_classification",  # 🏁 Classification mode
    results_path=results_path
)

automl.fit(X_train, y_train)

# --- Step 4: Predictions ---
y_pred_test = automl.predict(X_test)
y_pred_train = automl.predict(X_train)

# --- Step 5: Metrics ---
metrics = {
    "accuracy_test": accuracy_score(y_test, y_pred_test),
    "precision_test": precision_score(y_test, y_pred_test),
    "recall_test": recall_score(y_test, y_pred_test),
    "f1_test": f1_score(y_test, y_pred_test),
    "roc_auc_test": roc_auc_score(y_test, y_pred_test),
    "accuracy_train": accuracy_score(y_train, y_pred_train),
    "precision_train": precision_score(y_train, y_pred_train),
    "recall_train": recall_score(y_train, y_pred_train),
    "f1_train": f1_score(y_train, y_pred_train),
    "roc_auc_train": roc_auc_score(y_train, y_pred_train)
}

# --- Step 6: Define MLflow Pyfunc Wrapper ---
class MLJARWrapper(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        from supervised.automl import AutoML
        self.model = AutoML(results_path=results_path)

    def predict(self, context, model_input):
        return self.model.predict(model_input)

# --- Step 7: Save Model Folder ---
shutil.copytree(
    src=results_path,
    dst=model_path,
    dirs_exist_ok=True
)

# --- Step 8: Log into MLflow ---
with mlflow.start_run(run_name="Telecom_Churn_Classification"):
    # Params
    mlflow.log_params({
        "mode": "Compete",
        "time_limit_secs": 600,
        "eval_metric": "logloss",
        "task": "binary_classification",
        "framework": "MLJAR",
    })

    # Metrics
    mlflow.log_metrics(metrics)

    # Signature
    signature = infer_signature(X_train, y_pred_train)

    # Log model
    mlflow.pyfunc.log_model(
        artifact_path="model",
        python_model=MLJARWrapper(),
        signature=signature,
        code_path=[],
        registered_model_name="mljar_classification_model"
    )

print("✅ Classification model training complete and logged to MLflow successfully!")


2025/04/27 09:17:26 INFO mlflow.tracking.fluent: Experiment with name 'Telecom_Churn_Classification' does not exist. Creating a new experiment.


Linear algorithm was disabled.
AutoML directory: /Users/I353375/Downloads/MLOps/mlflow_server/mljar_classification/automl_results
The task is binary_classification with evaluation metric logloss
AutoML will use algorithms: ['Xgboost', 'LightGBM', 'CatBoost', 'Decision Tree', 'Extra Trees']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree logloss 0.396727 trained in 1.42 seconds
Adjust validation. Remove: 1_DecisionTree
Validation strategy: 5-fold CV Shuffle,Stratify
* Step simple_algorithms will try to check up to 3 models
1_DecisionTree logloss 0.385269 trained in 4.79 seconds
2_DecisionTree logloss 0.350166 trained in 5.01 se

Successfully registered model 'mljar_classification_model'.
2025/04/27 09:28:17 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: mljar_classification_model, version 1


✅ Classification model training complete and logged to MLflow successfully!


Created version '1' of model 'mljar_classification_model'.
